## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_data/WeatherPy_database.csv")
#school_data_to_load = os.path.join("Weather_Database", "WeatherPy_database.csv")
#city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,46.02,70,20,4.61,few clouds
1,1,New Norfolk,AU,-42.7826,147.0587,45.90,89,3,3.49,clear sky
2,2,Tha Mai,TH,12.6214,102.0048,82.06,85,68,2.35,broken clouds
3,3,Shingu,JP,33.7333,135.9833,62.15,89,100,7.81,light rain
4,4,Garmsar,IR,35.2182,52.3409,73.49,22,15,7.90,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? (°F) "))
max_temp = float(input("What is the maximum temperature you would like for your trip? (°F) "))

What is the minimum temperature you would like for your trip? (°F) 60
What is the maximum temperature you would like for your trip? (°F) 78


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Shingu,JP,33.7333,135.9833,62.15,89,100,7.81,light rain
4,4,Garmsar,IR,35.2182,52.3409,73.49,22,15,7.90,few clouds
5,5,Hermanus,ZA,-34.4187,19.2345,65.80,72,90,17.81,overcast clouds
6,6,Khash,IR,28.2211,61.2158,75.97,22,0,5.37,clear sky
7,7,Lucapa,AO,-8.4192,20.7447,74.91,58,95,3.67,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
712,712,Praya,CV,14.9215,-23.5087,75.74,69,0,13.80,clear sky
717,717,Apc,HU,47.7942,19.6943,61.79,71,39,2.53,scattered clouds
721,721,Gunjur,GM,13.2019,-16.7339,77.07,73,0,11.50,clear sky
722,722,Ambon,ID,-3.6954,128.1814,77.14,94,40,1.14,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                281
City                   281
Country                279
Lat                    281
Lng                    281
Max Temp               281
Humidity               281
Cloudiness             281
Wind Speed             281
Current Description    281
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()
clean_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Shingu,JP,33.7333,135.9833,62.15,89,100,7.81,light rain
4,4,Garmsar,IR,35.2182,52.3409,73.49,22,15,7.90,few clouds
5,5,Hermanus,ZA,-34.4187,19.2345,65.80,72,90,17.81,overcast clouds
6,6,Khash,IR,28.2211,61.2158,75.97,22,0,5.37,clear sky
7,7,Lucapa,AO,-8.4192,20.7447,74.91,58,95,3.67,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
712,712,Praya,CV,14.9215,-23.5087,75.74,69,0,13.80,clear sky
717,717,Apc,HU,47.7942,19.6943,61.79,71,39,2.53,scattered clouds
721,721,Gunjur,GM,13.2019,-16.7339,77.07,73,0,11.50,clear sky
722,722,Ambon,ID,-3.6954,128.1814,77.14,94,40,1.14,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Shingu,JP,62.15,light rain,33.7333,135.9833,
4,Garmsar,IR,73.49,few clouds,35.2182,52.3409,
5,Hermanus,ZA,65.80,overcast clouds,-34.4187,19.2345,
6,Khash,IR,75.97,clear sky,28.2211,61.2158,
7,Lucapa,AO,74.91,overcast clouds,-8.4192,20.7447,
8,Marsa Matruh,EG,65.30,clear sky,31.3525,27.2453,
9,Yuncheng,CN,62.49,overcast clouds,35.0231,110.9928,
12,Port Alfred,ZA,67.78,overcast clouds,-33.5906,26.8910,
18,Albany,US,76.03,clear sky,42.6001,-73.9662,
20,Jamestown,US,74.84,clear sky,42.0970,-79.2353,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters. Use the search term: "lodging" 
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(str(index) + "Hotel not found... skipping.")
        
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
     

18Hotel not found... skipping.
44Hotel not found... skipping.
115Hotel not found... skipping.
123Hotel not found... skipping.
193Hotel not found... skipping.
229Hotel not found... skipping.
240Hotel not found... skipping.
243Hotel not found... skipping.
250Hotel not found... skipping.
353Hotel not found... skipping.
497Hotel not found... skipping.
535Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [9]:
hotel_df.count()

City                   279
Country                279
Max Temp               279
Current Description    279
Lat                    279
Lng                    279
Hotel Name             279
dtype: int64

In [10]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Shingu,JP,62.15,light rain,33.7333,135.9833,Shingu UI Hotel
4,Garmsar,IR,73.49,few clouds,35.2182,52.3409,مجتمع مرصاد
5,Hermanus,ZA,65.80,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
6,Khash,IR,75.97,clear sky,28.2211,61.2158,خوابگاه آبجی محدثه
7,Lucapa,AO,74.91,overcast clouds,-8.4192,20.7447,Residência Ulisses Dos Santos
...,...,...,...,...,...,...,...
712,Praya,CV,75.74,clear sky,14.9215,-23.5087,LT Aparthotel
717,Apc,HU,61.79,scattered clouds,47.7942,19.6943,Német
721,Gunjur,GM,77.07,clear sky,13.2019,-16.7339,Jallo Kurundingoto Eco Tourist Camp
722,Ambon,ID,77.14,scattered clouds,-3.6954,128.1814,Swiss-Belhotel Ambon


In [11]:
#7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Shingu,JP,62.15,light rain,33.7333,135.9833,Shingu UI Hotel
4,Garmsar,IR,73.49,few clouds,35.2182,52.3409,مجتمع مرصاد
5,Hermanus,ZA,65.80,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
6,Khash,IR,75.97,clear sky,28.2211,61.2158,خوابگاه آبجی محدثه
7,Lucapa,AO,74.91,overcast clouds,-8.4192,20.7447,Residência Ulisses Dos Santos
...,...,...,...,...,...,...,...
712,Praya,CV,75.74,clear sky,14.9215,-23.5087,LT Aparthotel
717,Apc,HU,61.79,scattered clouds,47.7942,19.6943,Német
721,Gunjur,GM,77.07,clear sky,13.2019,-16.7339,Jallo Kurundingoto Eco Tourist Camp
722,Ambon,ID,77.14,scattered clouds,-3.6954,128.1814,Swiss-Belhotel Ambon


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a heatmap of temperature for the vacation spots and marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))